In [0]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#dir_path is set to the location containing empirical optimality ratios
#here it is set to the file path of the "EmpiricalOptimalityRatios" directory within "BatchOptimality"
dir_path = "/content/gdrive/My Drive/Active_Learning_Expected_Improvement_Maximization/BatchOptimality/EmpiricalOptimalityRatios/"


def ei_acquisition_trace_reduction (univ_covariance, num_pool_samples, num_training_samples, batch_size):
    """Runs batch-mode expected improvement maximization (EI) in order to obtain the resulting greedy trace/variance reduction
    #Arguments
        univ_covariance: Var(Y_{sample})
        num_pool_samples: number of sample points in the pool
        num_training_samples: number of sample points in the training set (throughout these experiments, this is set to 0; all points are considered for acquisition)
        batch_size: the number of queried points 
    #Returns
        The greedy trace reduction based on batch-mode EI
    """
    acq_ind = []
    orig_trace = np.trace(univ_covariance)
    for acq_num in range(batch_size):
        all_acq_values = np.zeros(num_pool_samples)
        for new_pt_ind in range(num_pool_samples):
            covariance_vector = univ_covariance[num_training_samples+new_pt_ind,:]
            #EI acquisition function in regression is equal to the squared norm of the covariance_vector divided by the predictive variance for the given point
            all_acq_values[new_pt_ind] = np.sum(np.square(covariance_vector))/(univ_covariance[num_training_samples+new_pt_ind, num_training_samples+new_pt_ind])
        sorted_top_ind = np.flip(np.argsort(all_acq_values))
        found_new_ind = False
        top_ind_ctr = -1
        while (found_new_ind == False):
            top_ind_ctr += 1
            new_top_ind = sorted_top_ind[top_ind_ctr]
            if new_top_ind not in acq_ind:
                acq_ind.append(new_top_ind)
                found_new_ind = True
        top_cov_vector = np.expand_dims(univ_covariance[num_training_samples+acq_ind[-1], :], axis=1)
        univ_covariance = univ_covariance - np.matmul(top_cov_vector, top_cov_vector.T)/univ_covariance[num_training_samples+acq_ind[-1], num_training_samples+acq_ind[-1]]
    return orig_trace - np.trace(univ_covariance)

In [0]:
#Function computing empirical optimality ratios

def greedy_opt_trace_reduction_exp (total_trials, num_samples_per_point, num_points, batch_size):
  """Compares the trace reduction from EI greedy batch assembly to the trace reduction resulting from querying the optimal batch of points under our assumptions
    #Arguments
      total_trials: the number of Var(Y_{sample}) that will be generated to compare the greedy trace reduction to the optimal trace reduction
      num_samples_per_point: the number of iid Gaussian samples corresponding to the model "predictions" for a given point
      num_points: |Y_{sample}|
      batch_size: number of queried points for which greedy/optimal trace reduction are compared
    #Returns
      An array of the optimality ratios: greedy trace reduction/optimal trace reduction, for all trials
  """
  all_opt_ratios = np.zeros(total_trials)
  for trial_ind in range(total_trials):
    a = np.random.normal(size=num_points * num_samples_per_point)
    b = a.reshape((num_points,num_samples_per_point))
    cov = np.cov(b) 
    orig_cov = cov + np.identity(cov.shape[0])*np.mean(np.diagonal(cov))*0.1
    orig_trace = np.trace(orig_cov)
    all_ind_subsets = list(itertools.combinations(set(range(0,num_points)), batch_size))
    all_trace_reductions = np.zeros(len(all_ind_subsets))
    subset_cntr = 0
    for ind_subset in all_ind_subsets:
      current_cov_matrix = orig_cov
      for ind in ind_subset:
        cov_vector = np.expand_dims(current_cov_matrix[ind, :], axis=1)
        current_cov_matrix = current_cov_matrix - np.matmul(cov_vector, cov_vector.T)/current_cov_matrix[ind,ind]
      all_trace_reductions[subset_cntr] = orig_trace - np.trace(current_cov_matrix)
      subset_cntr += 1
    opt_trace_reduction = np.amax(all_trace_reductions)
    #greedy trace reduction
    ei_trace_reduction = ei_acquisition_trace_reduction (orig_cov, num_points, 0, batch_size)
    opt_ratio = ei_trace_reduction/opt_trace_reduction
    all_opt_ratios[trial_ind] = opt_ratio
    #print('Finished one iteration')
  return all_opt_ratios



In [0]:
#Running and saving the results from empirical batch optimality experiments
bs2_3samp_opt_ratios = greedy_opt_trace_reduction_exp(200, 3, 200, 2)
print(bs2_3samp_opt_ratios)
np.save(dir_path + "BS2Samp3OptRatios.npy", bs2_3samp_opt_ratios)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


[1.         1.         0.99464395 0.99639867 0.98950971 0.99962984
 1.         1.         0.99179337 0.9977074  1.         0.99904544
 0.99888598 1.         0.99815781 1.         0.99178476 0.99252445
 1.         0.99372065 0.98888999 0.99742032 0.99863249 0.98926219
 1.         0.99490423 0.99534729 0.98971534 0.99070641 1.
 1.         1.         0.99260152 0.99346961 1.         0.99386418
 0.99417437 0.99862853 1.         1.         1.         1.
 1.         1.         0.99900203 1.         0.9967495  0.98678753
 0.9993827  0.99878313 0.98977856 1.         0.99455014 1.
 0.99373994 0.99625444 0.99417143 0.99796924 0.99150536 0.99862081
 0.99158374 0.99815206 0.99969336 1.         0.99927037 0.9990276
 1.         0.99656141 1.         0.99682621 0.99959243 0.99873202
 0.99733625 1.         0.99974464 1.         1.         0.99287127
 1.         0.99977467 0.99910917 0.99057182 1.         0.98984912
 1.         0.99596738 0.99349369 0.99661327 1.         0.99254591
 0.99342947 0.991345

In [0]:
#Computing Empirical Optimality Ratios for Batch Size 2
bs2_2samp_opt_ratios = greedy_opt_trace_reduction_exp(200, 2, 200, 2)
print(bs2_2samp_opt_ratios)
np.save(dir_path + "BS2Samp2OptRatios.npy", bs2_2samp_opt_ratios)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished on

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished on

In [0]:
bs2_50samp_opt_ratios = greedy_opt_trace_reduction_exp(200, 50, 200, 2)
print(bs2_50samp_opt_ratios)
np.save(dir_path + "BS2Samp50OptRatios.npy", bs2_50samp_opt_ratios)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished on

In [0]:
#Computing Empirical Optimality Ratios for Batch Size 5
bs5_2samp_opt_ratios = greedy_opt_trace_reduction_exp(200, 2, 30, 5)
print(bs5_2samp_opt_ratios)
np.save(dir_path + "BS5Samp2OptRatios.npy", bs5_2samp_opt_ratios)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished on

In [0]:
#Computing Empirical Optimality Ratios for Batch Size 5
bs5_3samp_opt_ratios = greedy_opt_trace_reduction_exp(200, 3, 30, 5)
print(bs5_3samp_opt_ratios)
np.save(dir_path + "BS5Samp3OptRatios.npy", bs5_3samp_opt_ratios)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


[1.         1.         1.         0.99933377 0.99979059 1.
 0.99997405 1.         1.         0.99897762 0.99995121 0.99789724
 0.99984383 0.99955928 1.         0.99994447 1.         0.99998137
 1.         0.99814078 0.99996843 1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         0.99995346 1.         1.         1.         0.99957397
 1.         1.         1.         1.         1.         1.
 1.         0.99984017 1.         0.9998818  1.         0.9974029
 1.         1.         1.         1.         1.         0.9993128
 1.         0.99916486 1.         1.         1.         1.
 0.99956976 0.99951104 0.9998666  1.         0.99887968 1.
 0.99832883 1.         0.99837636 1.         1.         1.
 1.         1.         0.99984212 1.         0.99995226 1.
 1.         1.         1.         0.99897464 0.9994573  1.
 1.         1.         0.99996931 0.99990041 1.         1.
 0.99978665 1.         1.         1.         1.         1.
 0.99903537 0.9991

In [0]:

bs5_50samp_opt_ratios = greedy_opt_trace_reduction_exp(200, 50, 30, 5)
print(bs5_50samp_opt_ratios)
np.save(dir_path + "BS5Samp50OptRatios.npy", bs5_50samp_opt_ratios)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished on

In [0]:
#Computing Empirical Optimality Ratios for Batch Size 10
bs10_2samp_opt_ratios = greedy_opt_trace_reduction_exp(200, 2, 20, 10)
print(bs10_2samp_opt_ratios)
np.save(dir_path + "BS10Samp2OptRatios.npy", bs10_2samp_opt_ratios)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished on

In [0]:
#Computing Empirical Optimality Ratios for Batch Size 10
bs10_3samp_opt_ratios = greedy_opt_trace_reduction_exp(200, 3, 20, 10)
print(bs10_3samp_opt_ratios)
np.save(dir_path + "BS10Samp3OptRatios.npy", bs10_3samp_opt_ratios)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


[1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         0.99999654 1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 0.99999873 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         0.99999546 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         

In [0]:

bs10_50samp_opt_ratios = greedy_opt_trace_reduction_exp(200, 50, 20, 10)
print(bs10_50samp_opt_ratios)
np.save(dir_path + "BS10Samp50OptRatios.npy", bs10_50samp_opt_ratios)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished one iteration
Finished on